# 含耦合器的两量子比特模拟器
*版权所有 (c) 2021 百度量子计算研究所，保留所有权利。*

## 内容概要

本教程将介绍如何使用量脉构建含耦合器件的两量子比特模拟器，分析其 $ZZ$ 耦合特征（多量子比特超导电路中一种常见的寄生耦合）,并且将演示在该结构模拟器 cross resonance 门的实现以及贝尔态的制备。本教程大纲如下：
+ 背景介绍
+ 准备工作
+ 构建系统的物理模型
+ 静态 $ZZ$ 耦合特征分析
+ Cross Resonance 门分析
+ 贝尔态制备
+ 总结

## 背景介绍

随着超导量子芯片中量子比特数目的持续增加，相邻量子比特的耦合会不可避免的产生串扰噪声，降低量子门的保真度，从而增加量子计算结果的错误率。为了有效减小量子芯片中的串扰噪声，业界提出了含耦合器件的超导电路架构并以此实现高保真度的两比特门 \[1\]。如下图所示，该架构由两个量子比特（Q1, Q2）与一个频率可调的耦合器（C）构成，$g_{1c}, g_{2c}$ 分别表示不同量子比特与耦合器之间的耦合强度，$g_{12}$ 是量子比特之间的直接耦合强度。

![tunable-coupler](figures/tunable-coupler-architectrue.png)

不同于经典的直接耦合架构，含耦合器件的超导电路架构可以通过调节耦合器的频率实现“打开”或“关闭”量子比特之间的耦合的操作，从而减少串扰噪声。目前，可调耦合架构已经被广泛应用于大型超导量子计算机的设计中 \[2, 3\]。

对于含耦合器件的超导电路架构，量脉提供了丰富的数值分析功能。本教程将演示如何分析含耦合器的两量子比特架构的$ZZ$ 耦合特征与该架构中 Cross resonance 门的特点，以及使用基于含耦合器件的超导电路架构的两量子比特模拟器模拟贝尔态的制备。

## 准备工作

成功安装量脉后，您可以按照本教程运行下面的量脉程序。要运行此教程，您需要从 Quanlse 和其它常用的 Python 库导入以下包：

In [ ]:
from math import pi
from numpy import linspace, array
from matplotlib import pyplot as plt

from Quanlse.Simulator.PulseSimQCQ import pulseSimQCQ
from Quanlse.Simulator import PulseModel
from Quanlse.Simulator.PulseSimQCQ import effectiveCoupling, pauliCoefficient
from Quanlse.Utils.Functions import basis, computationalBasisList
from Quanlse.Utils.Plot import plotBarGraph
from Quanlse.QOperation.FixedGate import CNOT
from Quanlse.QOperation.RotationGate import RX

## 构建系统的哈密顿量

在本节中，我们将演示如何使用量脉来构建含耦合器件的两量子比特的物理模型，该架构在实验室框架（lab frame）下的哈密顿量为 \[4\]：

$$ 
\hat{H}_{\rm sys}= \sum_{i=1,2,{\rm C}}\left(\omega_{i}\hat{a}_{i}^{\dagger}\hat{a}_{i} + \frac{\delta_{i}}{2}\hat{a}_{i}^{\dagger}\hat{a}_{i}^{\dagger}\hat{a}_{i}\hat{a}_{i}\right) + \sum_{i, j=1,2,{\rm C}}^{i \neq j}g_{ij}(\hat{a}_{i} + \hat{a}_{i}^{\dagger})(\hat{a}_{j} + \hat{a}_{j}^{\dagger}),
$$

其中 $1,2,{\rm C}$ 分别是两个量子比特以及耦合器的角标，$\hat{a}_{i}$ 与 $\hat{a}_{i}^{\dagger}$ 为相应元件的（量子比特以及耦合器）湮灭算符以及产生算符；$\omega_{i},\delta_{i}$ 分别表示各个元件对应的频率以及非谐性；$g_{ij}$ 表示不同元件之间的耦合强度。

接下来，我们将使用量脉来构建含耦合器件的两量子比特系统的哈密顿量，这将作为后续 $ZZ$ 耦合特征分析以及 CR 门实现的基础。首先定义模拟器需要的参数：

In [ ]:
# Parameters setting
couplerLevel = 3
qubitLevel = 4

subSysNum = 3
subSysLevel = [couplerLevel, qubitLevel, qubitLevel]  # [Coupler, Control Qubit, Target Qubit]

freqDict = {
    0: 6.3 * (2 * pi),  # Coupler frequency (GHz)
    1: 5.1 * (2 * pi),  # Control qubit frequency (GHz)
    2: 4.9 * (2 * pi)  # Target qubit frequency (GHz)
}

anharmDict = {
    0: 0.0 * (2 * pi),  # Coupler anharmonicity (GHz)
    1: -0.33 * (2 * pi),  # Control qubit anharmonicity (GHz)
    2: -0.33 * (2 * pi)  # Target qubit anharmonicity (GHz)
}

couplingMap = {
    (0, 1): 0.098 * (2 * pi),  # Coupling strength of coupler and control qubit (GHz)
    (0, 2): 0.083 * (2 * pi),  # Coupling strength of coupler and target qubit (GHz)
    (1, 2): 0.0025 * (2 * pi)  # Coupling strength of control qubit and target qubit (GHz)
}

接下来使用上述参数实例化一个 `PulseModel` 类的对象。

In [ ]:
exampleModel = PulseModel(subSysNum=subSysNum, sysLevel=subSysLevel, qubitFreq=freqDict, qubitAnharm=anharmDict,
                   couplingMap=couplingMap, frameMode='lab', dt=0.01)

构建好系统模型后，我们将演示如何使用分析工具来分析含耦合器的两量子比特架构 $ZZ$ 耦合特征 的以及该架构中 Cross resonance 门的实现。

**注意：量脉含耦合器的两量子比特架构分析工具目前仅支持特定的能级设置，其中耦合器的能级数量需设置为 3，量子比特的能级数量需设置为 4，耦合器的角标为 0，量子比特的角标为 1,2。**

## 静态 $ZZ$ 耦合特征分析

一般地，为了研究含耦合器的两量子比特架构自身性质，我们需要将模型转化为等效的两个量子比特的直接耦合模型，即将实验室系下的哈密顿量 $\hat{H}_{\rm lab}$  转化为等效模型哈密顿量 $\hat{H}_{\rm eff}$ \[4\]，

$$ 
\hat{H}_{\rm eff} = \sum_{i = 1, 2}\left(\tilde{\omega}_{i}\hat{a}_{i}^{\dagger}\hat{a}_{i} + \frac{\delta_{i}}{2}\hat{a}_{i}^{\dagger}\hat{a}_{i}^{\dagger}\hat{a}_{i}\hat{a}_{i}\right) + J(\hat{a}_{1} + \hat{a}_{1}^{\dagger})(\hat{a}_{2} + \hat{a}_{2}^{\dagger}),
$$

其中，$\tilde{\omega}_{i}$ 为量子比特的缀饰频率(dressed frequency)，$J$ 为两个量子比特的等效耦合强度（也称 $XY$ 强度）。等效耦合强度是可调耦合双量子比特架构中一个非常重要参数，我们可以通过调节耦合器的频率来改变等效耦合强度，从而实现“打开”或“关闭”两个量子比特之间的等效耦合的操作。

调用量脉 `Quanlse.Simulator.PulseSimQCQ` 模块中的 `effectiveCoupling()` 函数，输入实例化后的 `PulseModel` 对象可直接数值计算等效耦合强度的大小。

In [ ]:
# Obtain effective coupling strength (GHz)
print(f"Effective coupling strength: {(effectiveCoupling(exampleModel) / 2 / pi)} (GHz)")

等效耦合模型的哈密顿量在计算子空间（只考虑量子比特的基态和第一激发态）中可分解为两比特计算子空间中的多个泡利算符直积项的线性组合 \[4\]

$$ 
\hat{H}_{\rm sub} = \alpha_{ZI}\frac{ZI}{2} + \alpha_{IZ}\frac{IZ}{2} + \alpha_{ZZ}\frac{ZZ}{2}, 
$$

其中，$IZ, ZI, ZZ$ 为两比特空间中的泡利算符，第一个泡利算符作用于量子比特 1，而第二个泡利算符作用于量子比特 2，$\alpha_{ZI}, \alpha_{IZ}, \alpha_{ZZ}$ 为各个泡利算符的系数。

我们可以使用 `pauliCoefficient` 函数，输入实例化后的 `PulseModel` 对象，获取计算子空间中相应的泡利算符项的强度：

In [ ]:
print(f"Pauli coefficient: (GHz)")
print(f"  ZI: {pauliCoefficient(exampleModel)['ZI']}")
print(f"  IZ: {pauliCoefficient(exampleModel)['IZ']}")
print(f"  ZZ: {pauliCoefficient(exampleModel)['ZZ']}")

这里，$ZZ$ 相互作用项造成了量子比特之间的串扰噪声。在可调耦合双量子比特架构中，我们可通过调节耦合器的频率改变量子比特之间的等效耦合强度 $J$，从而消除 $ZZ$ 相互作用。接下来，我们将演示如何选取合适的耦合器频率从而消除量子比特之间的 $ZZ$ 串扰。通过改变耦合器频率，我们可以得到等效耦合强度 $J$ 以及 $ZZ$ 强度随耦合器频率变化的关系曲线。

In [ ]:
freqList = linspace(6.3, 8.8, 200) * (2 * pi)  # The range of different coupler frequencies, in 2 pi GHz

effectiveCouplingList = []
zzList = []

# Calculate effective coupling and ZZ interaction for different coupler frequencies
for freq in freqList:
    freqDict[0] = freq
    exampleModel.qubitFreq = freqDict
    effectiveCouplingList.append(effectiveCoupling(exampleModel))
    zzList.append(pauliCoefficient(exampleModel)['ZZ'])
    
plt.figure(figsize=[14, 6])
plt.subplot(121)
plt.plot(freqList / (2 * pi), abs(array(effectiveCouplingList) * 1e3))
plt.xlabel(r'$\omega_c$ (GHz)', fontsize=15)
plt.ylabel(r'$|\ J\ |$ (MHz)', fontsize=15)

plt.title('The absolute values of effective coupling strength', fontsize=15)

plt.subplot(122)
plt.plot(freqList / (2 * pi), abs(array(zzList)) * 1e6, 'r')
plt.xlabel(r'$\omega_c$ (GHz)', fontsize=15)
plt.ylabel(r'$|ZZ|$ (kHz)', fontsize=15)

plt.title('The absolute values of ZZ interaction strength', fontsize=15)
plt.show()

从图中我们可以看到，耦合器调节至 8.25 GHz 时，等效耦合强度 $J$ 与串扰噪声强度 $ZZ$ 均接近为 0，即消除了两个量子比特之间的串扰，这对于实现高保真度的量子门至关重要。

## Cross-resonance 门分析

在含耦合器的两量子比特架构中，不同的两量子比特门的实现方式不同：[iSWAP 门](https://quanlse.baidu.com/#/doc/tutorial-iswap) 与 [Control-Z 门](https://quanlse.baidu.com/#/doc/tutorial-cz) 可通过磁通调节耦合器频率或量子比特频率来实现，而 [Cross resonance 门](https://quanlse.baidu.com/#/doc/tutorial-cr)（简称 CR 门）需要通过在控制量子比特施加一个驱动脉冲实现 Cross resonance 效应（简称 CR 效应），从而实现门操作。本节将介绍使用量脉提供的分析工具分析含耦合器的两量子比特架构中的 CR 门。在可调耦合两量子比特架构中，实现 CR 门的哈密顿量为，

$$ 
\hat{H}_{\rm CR} = \hat{H}_{\rm sys} + \hat{H}_{\rm drive}, \\
\hat{H}_{\rm drive} = \Omega \cos(\omega_d t)(\hat{a}_{1} + \hat{a}_{1}^{\dagger}), 
$$

其中 $\hat{H}_{\rm drive}$ 为驱动项，$\Omega$ 为驱动振幅，$\omega_d$ 为驱动频率，这里我们设置控制量子比特为量子比特 1。同样的，我们将系统模型转化为等效的两个量子比特直接耦合模型。等效模型的哈密顿量在计算子空间可以表示成各个泡利算符的线性组合：

$$ 
\hat{H}_{\rm CR,sub} = \alpha_{ZI}\frac{ZI}{2} + \alpha_{IZ}\frac{IZ}{2} + \alpha_{ZZ}\frac{ZZ}{2} + \alpha_{IX}\frac{IX}{2} + \alpha_{IY}\frac{IY}{2} + \alpha_{ZX}\frac{ZX}{2} + \alpha_{ZY}\frac{ZY}{2}.
$$

其中，$\alpha_{ZI}, \alpha_{IZ}, \alpha_{ZZ}, \alpha_{IX}, \alpha_{IY}, \alpha_{ZX}, \alpha_{ZY}$ 为对应泡利算符强度。对于 CR 门来说，$ZX$ 项为门实现所需要的项，$ZZ, IX, ZY, IY$ 均为噪声项。通过调整器件参数或驱动脉冲振幅大小，可以改变不同泡利算符项的大小，从而实现噪声的消除。下面我们将演示改变驱动脉冲幅值 `drivingAmp`，从而改变 CR 效应中不同泡利算符的强度 \[4\]。

我们可以使用 `pauliCoefficient()` 函数，输入实例化后的 `PulseModel` 对象以及驱动脉冲的振幅大小 `drivingAmp` 来获取计算子空间相应的泡利算符项的强度。

In [ ]:
drivingAmpRange =linspace(0, 0.10, 50) * 2 * pi
coeffs = []

for drivingAmp in drivingAmpRange:
    coeffs.append(pauliCoefficient(exampleModel, drivingAmp))
    
ZI = array([coeff['ZI'] for coeff in coeffs])
IZ = array([coeff['IZ'] for coeff in coeffs])
ZZ = array([coeff['ZZ'] for coeff in coeffs])
ZX = array([coeff['ZX'] for coeff in coeffs])
ZY = array([coeff['ZY'] for coeff in coeffs])
IX = array([coeff['IX'] for coeff in coeffs])
IY = array([coeff['IY'] for coeff in coeffs])

In [ ]:
# Define unit
unit = 1e3 / (2 * pi)

plt.figure(figsize=[14, 6])
plt.subplot(121)
[zx, ix] = plt.plot(drivingAmpRange * unit, ZX * 1e3, drivingAmpRange * unit, IX * 1e3)
plt.legend([zx, ix], ['ZX', 'IX'], fontsize=12)
plt.xlabel('Driving amplitude (MHz)', fontsize=15)
plt.ylabel('Interaction strength (MHz)', fontsize=15)
plt.title('Interaction strength vs Driving amplitude', fontsize=15)

plt.subplot(122)
[zz, zy, iz, iy] = plt.plot(drivingAmpRange * unit, ZZ * 1e6, drivingAmpRange * unit, ZY * 1e6, drivingAmpRange * unit, 
                            IZ * 1e6, drivingAmpRange * unit, IY * 1e6)
plt.legend([zz, zy, iz, iy], ['ZZ', 'ZY', 'IZ', 'IY'], fontsize=12)
plt.xlabel('Driving amplitude (MHz)', fontsize=15)
plt.ylabel('Interaction strength (kHz)', fontsize=15)
plt.title('Interaction strength vs Driving amplitude', fontsize=15)

plt.show()

由上图可知，随着驱动脉冲振幅变化，不同泡利算符的强度也在发生变化，通过选取高强度的 $ZX$ 以及低噪声强度对应的脉冲振幅，可实现高保真度的 CR 门。

## 贝尔态制备

下面我们将演示如何利用含耦合器的双量子比特架构来制备贝尔态。 贝尔态是指两个量子比特的最大纠缠态 \[5\]，其中一个例子是量子态 $ \frac{1}{\sqrt{2}} ( | 00 \rangle + | 11 \rangle )$。贝尔态可由如下电路生成，其中包括 $ X_{\pi / 2} $，即沿 $ x $ 轴的 $ \pi / 2 $ 的旋转门，和两量子比特 CNOT 门。

![Bell-state-CNOT](figures/Bell-state-CNOT.png)

这里，我们需要使用量脉中预先定义、具有默认配置的 `PulseModel` 对象。为简便起见，默认配置中耦合器和量子比特的能级数量均设置为 2，耦合器的角标为 0，量子比特的角标分别为 1, 2。我们首先通过调用 `pulseSimQCQ` 实例化一个 `PulseModel` 类的对象。

In [ ]:
model = pulseSimQCQ(dt=0.01, frameMode='lab')

对于这个具有默认配置的含耦合器双量子比特架构，量脉提供了已标定好的单比特和两比特门操作。我们可以调用这些门操作定义制备贝尔态所需的量子电路。

In [ ]:
# RX(pi / 2) gate
RX(pi / 2)(model.Q[1])

# CNOT: 1 -> 2
CNOT(model.Q[1], model.Q[2])

上面定义的量子电路的脉冲序列可以通过调用函数 `model.schedule` 生成。我们可以使用 `plot()` 来可视化量子电路对应的脉冲任务。图中 `uWave1` 和 `uWave2` 分别为施加于控制量子比特和目标量子比特的脉冲信号。耦合器上没有施加脉冲，因此我们略去该频道。这里，我们使用了 Echo-CR 方案实现 CR 门 \[6\]，然后使用单量子比特门以及 CR 门构建了一个 CNOT门（详细内容可见 [Cross resonance 门](https://quanlse.baidu.com/#/doc/tutorial-cr) 教程）。

In [ ]:
scheJob = model.schedule()
scheJob.plot()

设置系统的初始态为 $| \psi \rangle = | 000 \rangle$，其中三个数字分别对应耦合器，控制量子比特和目标量子比特的状态。然后运行模拟，并且画出每个测量结果的概率分布。

In [ ]:
# Run the simulation
psi0 = basis(8, 0)
res = model.simulate(state0=psi0, job=scheJob)

psi1 = res[0]['state']

# Plot the result
popList = [abs(item ** 2) for item in res[0]['state'].T[0]]
basisList = computationalBasisList(3, 2)
plotBarGraph(basisList, popList, "Result", "Outcome", "Population")

由上图可见，测量结果主要是 $ | 000 \rangle $ 和 $ | 011 \rangle $ 态。由此可知我们得到了一个保真度约 80% 的近似贝尔态。

## 总结

本教程介绍了如何使用量脉的工具分析含耦合器的双量子比特架构中的$ZZ$ 耦合以及 CR 门的特征，并演示了基于含耦合器双量子比特架构的贝尔态的制备。用户可以点击这个链接 [tutorial-tunable-coupling-architectrue-two-qubit-simulator-cn.ipynb](https://github.com/baidu/Quanlse/blob/main/Tutorial/CN/tutorial-two-qubit-simulator-with-coupler-architecture-cn.ipynb) 跳转到此 Jupyter Notebook 文档相应的 GitHub 页面来获取相关代码。我们推荐用户使用不同于本教程的参数来分析含耦合器两量子比特架构，模拟两量子比特门的实现，并开发适用范围更广的分析工具。

## 参考文献

\[1\] [Yan Fei, et al. "Tunable coupling scheme for implementing high-fidelity two-qubit gates." *Physical Review Applied* 10.5 (2018): 054062.](https://journals.aps.org/prapplied/abstract/10.1103/PhysRevApplied.10.054062)

\[2\] [Arute, Frank, et al. "Quantum supremacy using a programmable superconducting processor." *Nature* 574.7779 (2019): 505-510.](https://www.nature.com/articles/s41586-019-1666-5)

\[3\] [Wu, Yulin, et al. "Strong quantum computational advantage using a superconducting quantum processor." *arXiv preprint arXiv:2106.14734* (2021).](https://arxiv.org/abs/2106.14734)

\[4\] [Magesan, Easwar, and Jay M. Gambetta. "Effective Hamiltonian models of the cross-resonance gate." *Physical Review A* 101.5 (2020): 052308.](https://link.aps.org/doi/10.1103/PhysRevA.101.052308)

\[5\] ["Bell state" *Wikipedia*.](https://en.wikipedia.org/wiki/Bell_state)

\[6\] [Córcoles, Antonio D., et al. "Process verification of two-qubit quantum gates by randomized benchmarking." *Physical Review A* 87.3 (2013): 030301.](https://journals.aps.org/pra/abstract/10.1103/PhysRevA.87.030301)